In [5]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

from qiskit import *
from qiskit.providers.ibmq import least_busy
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute
from qiskit.quantum_info import Operator

from qiskit.tools.visualization import plot_histogram
from IPython.display import display, Math, Latex

import pennylane as qml

# QAOA for MaxCut

In this notebook we will discuss a Quantum Approximate Optimization Algorithm *(QAOA)* for the MaxCut problem, which is combinatorial problem that cannot be computed in polynomial time, unless **P = NP**.

So let G be a graph on $n$ vertices. Given $\emptyset \neq S \subset V(G)$ we define the cut $\delta(S)$ as the set of edges connecting $S$ to its complement $\bar S$, and $\mu(G)$ denotes the size of the optimal cut for the given graph. Moreover, our focus here is to determine a cut that maximizes the number of cut edges, i.e, find the largest value for $|E(S, \bar S)|$, given that

$$
\begin{align*}
    |E(S, \bar S)| = \{(u, v)\text{ }|\text{ }u \in S, v \in \bar S\}
\end{align*}
$$


Now, let $S \subseteq V(G)$, then $x_s$ denotes the $\mathbb{Z}_2^n$-characteristic vector of $S \in \mathbb{R}^{|V(G)|}$ and $x_{\delta(S)}$ the $\mathbb{Z}_2^n$-characteristic vector of $\delta(S) \in \mathbb{R}^{E(G)}$. Then, we can write our problem as linear program given by

$$
\begin{align*}
\text{max} &\quad \frac{1}{2}\sum_{ij\text{ }\in\text{ }E(G)} w_{ij}(1 - x_i x_j)\\
\text{subject to} &\quad x_i \in \{-1, 1\}
\end{align*}
$$


Which is a very natural formulation for our Maxcut, where we also assume that the edges of the graph can be weighted.

## A Quantum Approximation Approach 

In order to quantumly encode our objective function, we will define a *Cost Hamiltonian* such that

$$
\begin{align*}
H_c|x\rangle = C(x)|x\rangle
\end{align*}
$$

Where $|x\rangle$ is a computational basis vector, which can be seen as an eigenvalue problem, defined as $T(v) = \lambda v$, where the edge $C_{ij}$ has eigenvalue 1 if and only if the $i$th and $j$th qubits have different measurements results, which represents separete partitions of our sets $S$ and $\bar S$.

And it can be written as a diagonal hamiltonian in the computational basis,

$$
\begin{align*}
H_c = \sum_{x \in\{0, 1\}^n} C(x)|x\rangle\langle x|
\end{align*}
$$

We can rewrite it expanding into Pauli-$Z$ operators, by substituting for each vector $x_i \in \{0, 1\}^n$ the matrix $x_i \rightarrow \frac{1}{2}(1 - Z_i)$, where $Z_i$ is the $Z$ operator that acts on qubit $i$. And with this assigment, we have the final equation that defines our hamiltonian in terms of the edges of the graph.

$$
\begin{align*}
H_c = \frac{1}{2}\sum_{(i, j) \in E(G)} (1 - \sigma_z^i\sigma_z^j)
\end{align*}
$$

And finally we are able to define our unitary operator that will be used in our QAOA circuit,

$$
\begin{align*}
U(H_c, \gamma) = \exp(-i\gamma H_c) = \prod^m_{\alpha = 1} e^{-i\gamma C_{\alpha}}\\
\text{          where } \gamma \in [0, 2\pi]
\end{align*}
$$

Where $\gamma \in [0, 2\pi]$

Moreover, we define our mixing Hamiltonian $H_m$ as

$$
H_m = \sum_{i = 0}^n \sigma_i^x
$$

Where $\sigma_i^x$ is the Pauli-$X$ gate acting on the $i$th qubit. And we define the $\beta$ dependent $U(H_m, \beta)$ unitary operator as it follows.

$$
\begin{align*}
U(H_m, \beta) = \exp(-i\beta H_m) = \prod^n_{\alpha = 1} e^{-i\beta X_{\alpha}}\\
\text{          where } \beta \in [0, \pi]
\end{align*}
$$

In [13]:
n_wires = 5

#Butterfly Graph
graph = [(0, 1), (0, 2), (1, 2), (2, 3), (2, 4), (3, 4)]

#Cost Hamiltonian
def U_Hc(beta):
    for wire in range(n_wires):
        qml.RX(2 * beta, wires=wire)


#Mixing Hamiltonian
def U_Hm(gamma):
    for edge in graph:
        wire1 = edge[0]
        wire2 = edge[1]
        qml.CNOT(wires=[wire1, wire2])
        qml.RZ(gamma, wires=wire2)
        qml.CNOT(wires=[wire1, wire2])

#Computational Basis Measurement
def measurement(wires):
    n_wires = len(wires)
    return qml.Hermitian(np.diag(range(2 ** n_wires)), wires=wires)

### References

[[1]](https://arxiv.org/pdf/1411.4028.pdf) A Quantum Approximate Optimization Algorithm, Farhi, Goldstone, and Gutmann (2014)

[[2]](https://pennylane.ai/qml/demos/tutorial_qaoa_maxcut.html) PennyLane QAOA for MaxCut

[[3]](https://qiskit.org/textbook/ch-applications/qaoa.html) Solving combinatorial optimization problems using QAOA